# Acquire MVP

For the first iteration of your model, use only square feet of the home, number of bedrooms, and number of bathrooms to estimate the property's assessed value, taxvaluedollarcnt. You can expand this to other fields after you have completed an mvp (minimally viable product).

### Imports and SQL Query

Import packages, database credentials, and get_connection function.

In [1]:
import pandas as pd
import numpy as np

import os

from env import host, user, password

def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

I'll also set my SQL query up here rather than have it crowd the notebook further down, for readability's sake.

In [2]:
sql_query = '''
            SELECT properties_2017.parcelid, properties_2017.id, bathroomcnt, bedroomcnt, calculatedbathnbr, calculatedfinishedsquarefeet, fips, latitude, longitude, regionidcounty, roomcnt, yearbuilt, taxvaluedollarcnt, assessmentyear, propertycountylandusecode, propertylandusetypeid
            FROM properties_2017
            JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
            WHERE transactiondate BETWEEN '2017-05-01' AND '2017-06-30'
            AND propertylandusetypeid = '261' OR '262' OR '263' OR '264' OR '268' OR '273' OR '274' OR '275' OR '276' OR '279';
            '''

In [7]:
sql_query = ''''
            SELECT properties_2017.parcelid, properties_2017.id, bathroomcnt, bedroomcnt, calculatedbathnbr, calculatedfinishedsquarefeet, fips, latitude, longitude, regionidcounty, roomcnt, yearbuilt, taxvaluedollarcnt, taxamount, assessmentyear, propertycountylandusecode, propertylandusetypeid, transactiondate
            FROM properties_2017
            JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
            WHERE transactiondate BETWEEN '2017-05-01' AND '2017-06-30'
            AND propertylandusetypeid = '261' OR '262' OR '263' OR '264' OR '268' OR '273' OR '274' OR '275' OR '276' OR '279';
            '''

From https://help.rentingwell.com/article/multi-unit-vs-single-unit/:

> A **multi-unit property** is a rental property that has been divided into multiple units that are rented independently to different tenants. A duplex, a triplex, or an apartment building would all be multi-unit properties.

> A **single-unit property** is a rental property that is rented as a single entity. A condo, a townhouse, or a vacation rental would typically be single-unit properties.

We only want to predict on single-unit properties, so I will select only single-unit properties from the Zillow data in SQL:
 * **261 -** Single Family Residential
 * **262 -** Rural Residence
 * **263 -** Mobile Home
 * **264 -** Townhouse
 * **268 -** Row House
 * **273 -** Bungalow
 * **274 -** Zero Lot Line
 * **275 -** Manufactured, Modular, Prefabricated Homes
 * **276 -** Patio Home
 * **279 -** Inferred Single Family Residential
 
---

### Acquire Data

Read the data from SQL into a dataframe and save that dataframe as a .csv locally.

In [12]:
filename = "zillow_df.csv"
if os.path.isfile(filename):
    df = pd.read_csv(filename)
    #drop second index column
    df = df.drop(columns=['Unnamed: 0'])
else:
    # read the SQL query into a dataframe
    df = pd.read_sql(sql_query, get_connection('zillow'))
    #drop second index column
    df = df.drop(columns=['Unnamed: 0'])
    # Write that dataframe to disk for later. Called "caching" the data for later.
    df.to_file(filename)
df.head(3)

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,latitude,longitude,regionidcounty,roomcnt,yearbuilt,taxvaluedollarcnt,assessmentyear,propertycountylandusecode,propertylandusetypeid
0,14297519,1727539,3.5,4.0,3.5,3100.0,6059.0,33634931.0,-117869207.0,1286.0,0.0,1998.0,1023282.0,2016.0,122,261.0
1,17052889,1387261,1.0,2.0,1.0,1465.0,6111.0,34449266.0,-119281531.0,2061.0,5.0,1967.0,464000.0,2016.0,1110,261.0
2,14186244,11677,2.0,3.0,2.0,1243.0,6059.0,33886168.0,-117823170.0,1286.0,6.0,1962.0,564778.0,2016.0,122,261.0


In [13]:
df.columns

Index(['parcelid', 'id', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'fips', 'latitude', 'longitude',
       'regionidcounty', 'roomcnt', 'yearbuilt', 'taxvaluedollarcnt',
       'assessmentyear', 'propertycountylandusecode', 'propertylandusetypeid'],
      dtype='object')

### Summarize Data

In [14]:
print(f"The shape of the original dataframe:\n {df.shape[0]} rows and {df.shape[1]} columns.")

The shape of the original dataframe:
 77614 rows and 16 columns.


In [15]:
# I am running this line iteratively throughout acquire.
df.isnull().sum()

parcelid                          0
id                                0
bathroomcnt                      34
bedroomcnt                       34
calculatedbathnbr               650
calculatedfinishedsquarefeet    235
fips                             34
latitude                         34
longitude                        34
regionidcounty                   34
roomcnt                          34
yearbuilt                       304
taxvaluedollarcnt                35
assessmentyear                   34
propertycountylandusecode        34
propertylandusetypeid            34
dtype: int64

**Iteration 1**

I'm going to go back to my SQL query and edit it to select only the columns that I want. I won't select columns that contain mostly null values.

calculatedbathnbr has 55 null values, so I am going to recreate that variable on the next interation of my MVP with a combination of bathroomcnt and bedroomcnt. For now, I will leave it and impute the nulls with bathroom + bedroomcnt.

If I have time in a future iteration I might use longitude and latitude with Geopy.

Although regionidzip has only 10 missing values, I'm choosing to leave it out for now because I don't think imputing a median will give useful or near-accurate values, and for location I can use county, as that's what we've been asked to compare anyway.

I'm also not selecting columns that have similar or duplicate information.

---

**Iteration 2**

I see that there are many columns that have 34 null values, so I will drop those null-heavy rows.

For this, I will continue to the next stage of the pipeline, Prepare, in the prepare_kwame.ipynb notebook.

Before I do that, however, I must create an acquire_kwame.py file so that I can make the code in this notebook into functions that I can import into prepare_kwame.ipynb and then into prepare_kwame.py as well. I'll also add the code to cache the .csv in the .py file.

In the next iteration I can summarize the data in more ways and show distributions. Remember not to compare variables until the train, validate, test split.

---